In [6]:
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community import louvain_communities, modularity


In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import networkx as nx
import pandas as pd
import time 

CLIENT_ID     = "9698e462a6bb493ea901b6856cab619f"
CLIENT_SECRET = "dd9b9a7ecb06457bad00b54cec5a0b26"

auth_manager = SpotifyClientCredentials (
            client_id     = CLIENT_ID ,
            client_secret = CLIENT_SECRET )

sp = spotipy.Spotify( auth_manager = auth_manager )

In [8]:
def num_common_nodes(*arg):
    """
    Return the number of common nodes between a set of graphs.

    :param arg: (an undetermined number of) networkx graphs.
    :return: an integer, number of common nodes.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    common_nodes = set(arg[0].nodes)
    for graph in arg[1:]:
        nodes = set(graph.nodes)
        common_nodes = common_nodes.intersection(nodes)

    return len(common_nodes)
    # ----------------- END OF FUNCTION --------------------- #

In [9]:
def get_degree_distribution(g: nx.Graph) -> dict:
    """
    Get the degree distribution of the graph.

    :param g: networkx graph.
    :return: dictionary with degree distribution (keys are degrees, values are number of occurrences).
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    degree_count = nx.degree_histogram(g)
    degree_distribution = {degree: count for degree, count in enumerate(degree_count)}
    return degree_distribution
    # ----------------- END OF FUNCTION --------------------- #

In [10]:
def get_k_most_central(g: nx.Graph, metric: str, num_nodes: int) -> list:
    """
    Get the k most central nodes in the graph.

    :param g: networkx graph.
    :param metric: centrality metric. Can be (at least) 'degree', 'betweenness', 'closeness' or 'eigenvector'.
    :param num_nodes: number of nodes to return.
    :return: list with the top num_nodes nodes with the specified centrality.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    if metric == "degree":
        most_central = sorted(nx.degree_centrality(g).items(), key=lambda item: item[1], reverse=True)[:num_nodes]
    elif metric == "betweeness":
        most_central = sorted(nx.betweenness_centrality(g).items(), key=lambda item: item[1], reverse=True)[:num_nodes]
    elif metric == "closeness":
        most_central = sorted(nx.closeness_centrality(g).items(), key=lambda item: item[1], reverse=True)[:num_nodes]
    elif metric == "eigenvector":
        most_central = sorted(nx.eigenvector_centrality(g).items(), key=lambda item: item[1], reverse=True)[:num_nodes]
    elif metric == "page_rank":
        most_central = sorted(nx.pagerank(g).items(), key=lambda item: item[1], reverse=True)[:num_nodes]
    
    return most_central
    # ----------------- END OF FUNCTION --------------------- #

In [11]:
def find_cliques(g: nx.Graph, min_size_clique: int) -> tuple:
    """
    Find cliques in the graph g with size at least min_size_clique.

    :param g: networkx graph.
    :param min_size_clique: minimum size of the cliques to find.
    :return: two-element tuple, list of cliques (each clique is a list of nodes) and
        list of nodes in any of the cliques.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    cliques = [clic for clic in nx.find_cliques(g) if len(clic) > min_size_clique]
    clique_nodes = list(set(node for clique in cliques for node in clique))

    return cliques, clique_nodes
    # ----------------- END OF FUNCTION --------------------- #

In [12]:
import networkx as nx
from networkx.algorithms.community.quality import is_partition, modularity

def detect_communities(g: nx.Graph, method: str) -> tuple:
    """
    Detect communities in the graph g using the specified method.

    :param g: a networkx graph.
    :param method: string with the name of the method to use. Can be (at least) 'givarn-newman' or 'louvain'.
    :return: two-element tuple, list of communities (each community is a list of nodes) and modularity of the partition.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    if method == "girvan_newman":
        communities = [comuniti[0] for comuniti in list(girvan_newman(Graph_B))]
    elif method == "louvain":
        communities = louvain_communities(g)
    else:
        raise ValueError("Invalid method name.")

    if not is_partition(g, communities):
        raise ValueError("Communities do not form a valid partition.")

    modularity_partition = modularity(g, communities)
    return communities, modularity_partition
    # ----------------- END OF FUNCTION --------------------- #


In [13]:
def detect_communities(g: nx.Graph, method: str) -> tuple:
    """
    Detect communities in the graph g using the specified method.

    :param g: a networkx graph.
    :param method: string with the name of the method to use. Can be (at least) 'givarn-newman' or 'louvain'.
    :return: two-element tuple, list of communities (each community is a list of nodes) and modularity of the partition.
    """
    # ------- IMPLEMENT HERE THE BODY OF THE FUNCTION ------- #
    if method == "girvan_newman":
        communities = list(girvan_newman(g))
        
    elif method == "louvain":
        communities = louvain_communities(g)
        
    modularity_partition = modularity(g, communities)

    return (communities, modularity_partition)

    # ----------------- END OF FUNCTION --------------------- #

In [21]:
Graph_B  = nx.read_graphml('/Users/nbiescas/Desktop/Graphs/Undirected_graph_B.graphml.xml')
Graph_D  = nx.read_graphml('/Users/nbiescas/Desktop/Graphs/Graphs_data/Graph_D.graphml.xml')

In [14]:
Graph_B_und = nx.read_graphml('/Users/nbiescas/Desktop/Graphs/Graphs_data/Undirected_graph_B.graphml')
Graph_D_und = nx.read_graphml('/Users/nbiescas/Desktop/Graphs/Graphs_data/Undirected_graph_D.graphml')

In [15]:
GB = find_cliques(Graph_B_und, min_size_clique = 7)
GD = find_cliques(Graph_D_und, min_size_clique = 10)
print(len(GB[0]))
print(len(GD[0]))

GB_art = GB[1]

0
0


In [16]:
art = {}
for artists in GB_art:
    artist = sp.artist(artists)
    name = artist['name']
    followers = artist['followers']['total']
    popularity = artist['popularity']
    genres = artist['genres']
    art[name] = {"followers": followers, "popularity": popularity, "genres": genres}

art = pd.DataFrame(art).T

max_followers = art['followers'].max()
min_followers = art['followers'].min()
range_followers = max_followers - min_followers

# Find the minimum popularity
min_popularity = art['popularity'].min()
max_popularity = art['popularity'].max()
range_popularity = max_popularity - min_popularity



# Find the intersection of genres
all_genres = set()

for genres_list in art['genres']:
    for genre in genres_list:
        all_genres.add(genre)

count = sum(1 for genre in all_genres if 'hip hop' in genre or 'rap' in genre)

print(min_followers)
print(max_followers)
print(range_followers)

print(min_popularity)
print(max_popularity)
print(range_popularity)

print(all_genres)
print(len(all_genres))

print(count)

KeyError: 'followers'

In [ ]:
if __name__ == '__main__':
    # ------- IMPLEMENT HERE THE MAIN FOR THIS SESSION ------- #
    #common_nodes = num_common_nodes(...)
    #deg_dist = get_degree_distribution(...)
    #most_central = get_k_most_central(...)
    #cliques, cliques_nodes = find_cliques(...)
    #communities, modularity = detect_communities(...)
    # ------------------- END OF MAIN ------------------------ #